In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType

In [2]:
SUFFIX = '_MIN_FOUR_VARIABLES_DATASET_NR_'#'_recategorized'
#FILE_TO_LOAD = 'CHES2019_experts.csv' # 'CHES2019_experts_recategorized_4l3c4r.csv' #' CHES2019_experts_recategorized.csv'#'CHES2019_experts.csv', 
FOLDER_PREFIX = '' #'base_data/' #'recategorized_4l3c4r/' # 
FOLDER_IMPUTE_PREFIX = 'imputed_data/'
relevant_cols_EVS_independent = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle']

## PCA

In [8]:
# load dataset
# dataset is complete with full imputation
dataset = pd.read_csv('CHES2019_experts_imputed_cart.csv')
dataset = dataset[['econ_interven','environment','redistribution','civlib_laworder','immigrate_policy','sociallifestyle']]
EVS_dataset = pd.read_csv('EVS_data_imputed.csv')
EVS_dataset = EVS_dataset[relevant_cols_EVS_independent]
X_std = StandardScaler().fit_transform(dataset)
EVS_dataset_std = StandardScaler().fit_transform(EVS_dataset)



In [9]:
pca = PCA(n_components=2)
expert_pca_pred = pca.fit_transform(X_std)

In [10]:
pca.components_

array([[ 0.28876877,  0.4217626 ,  0.33932188,  0.46100875,  0.46718632,
         0.43908705],
       [ 0.6574434 , -0.05819727,  0.58798943, -0.26555983, -0.22615188,
        -0.31142093]])

In [11]:
print(sum(pca.explained_variance_ratio_))

0.8011538669561444


In [12]:
EVS_pca = pd.DataFrame(pca.transform(EVS_dataset_std),columns=['PC1', 'PC2'])

In [13]:
pd.concat([EVS_dataset, EVS_pca], axis=1).to_csv('result_application_PCA_to_voter_data.csv')

In [14]:
expert_pca = pd.DataFrame(expert_pca_pred, columns=['PC1', 'PC2'])
pd.concat([dataset, expert_pca], axis=1).to_csv('expert_PCA_data.csv')

## Export PCA as onnx

In [15]:
initial_type = [('float_input', FloatTensorType([None, dataset.shape[1]]))]
model_onnx = convert_sklearn(pca, initial_types=initial_type)
meta =  model_onnx.metadata_props.add()
with open("pca.onnx", "wb") as f:
    f.write( model_onnx.SerializeToString())